# Liquid Types, Revisited
## Nathan Taylor (ntaylor@cs), Sam Thomas (sgt@cs)

### C S 395T: The Model Checking Paradigm, 24 April 2023

# A type paper in a model-checking class?

* In this class we've talked about propositions, proofs, counterproofs... does this feel like a different world than the typed programming you know from languages like Java?

* TODO Guess what!!! propositions=types, program=proofs, counterproofs=...?? type errors???

* Type semantics relates a class of inputs  (all possible valid envioronments) to a class of outputs (all possible instantiations of a type) - feels like abstraction or “modulo”!!!

* This means we can use the vocabulary of model-checking in a type context.  This is great because type systems give us lots of nice guarantees: soundness, reconstruction, termination, etc...

## Type systems are great for proving properties of our programs...


What well-typed argument can we pass to `avg()` that will cause this program to break?

In [ ]:
def avg(xs):
    return sum(xs) // len(xs)

assert avg([1,2,3]) == 2
assert avg([1]) == 1
avg([])

## Do well-typed programs never go wrong?

Let's annotate our program with types, like good programmers.  The premise of a type system is that it rejects invalid programs: can you think of input to `avg()` that will still cause an exception to be thrown?

In [42]:
def avg(xs: list[int]) -> int:
    """ Computes the average of a list of ints."""
    return sum(xs) // len(xs)

assert avg([1,2,3]) == 2
assert avg([1]) == 1


## Extending the expressivity of a type system

* Somebody in this room once said, "An abstraction is a constrained proof system"
    * a richer _constraint domain_ in our proof system lets us encode richer abstractions.

What sort of constraint domain would let us encode the type of a list that is provably nonempty (and thus avoiding the division by zero exception), while still maintaining:
* Automation - no proof
* etc

---

Our goal: 
```python
def avg(l: {some proposition that will reject the empty list} ) -> int:
    return sum(l.l) // len(l.l)
avg([])      # should produce a type error
avg([1,2,3]) # should not produce a type error
```

In [ ]:
# Note: this slide imports a bunch of stuff for the peano-indexed Vec example
from __future__ import annotations
from typing import Any, Generic, TypeVar
from dataclasses import dataclass
N,T = TypeVar("N", bound="Nat"), TypeVar("T")

## Non-solution 1: Keep the proof system; enrich the List type

A standard solution is to _index_ the List type with an additional type parameter which represents its length.

This means we need an encoding of the numbers _in the type system_, since it's going to be a type argument.

(Fun fact: this is exactly what we did in Coq in the first class of grad PL last year)

In [ ]:
# An encoding of the natural numbers, entirely in the type system
# (notice the S and Z classes have no fields!)
class Nat: pass
class Z(Nat): pass              # Z is zero
class S(Nat, Generic[N]): pass  # S[N] is the successor of some other nat N

In [ ]:
# Note: `Two` and `Four` are typedefs for some subtypes of Nat, not 
# program-level terms with values we can compute on at runtime!
Two: type  = S[S[Z]]
Four: type = S[S[Two]]

The two constructors `nil` and `cons` enforce the invariant that the typelevel value of `N` always conforms to the length of the list.

All the benefits we get from our type system remain!

In [43]:
# Here's our vector of N elements, all of type T.
@dataclass
class Vec(Generic[N,T]):
    l: list[T]
        
def nil() -> Vec[Z, T]: return Vec([])
def cons(t: T, l: Vec[N, T]) -> Vec[S[N], T]: return Vec([t] + l.l)

# The type system knows the lengths of these lists!
empty: Vec[Z, int] = nil()
one_three:  Vec[Two, int] = cons(1, cons(3, nil()))

And critically, avg() will statically-reject the empty list (since the type says the length must be the successor of some other Nat; this isn't true for zero!)

Unfortunately, the type system wasn't really meant to do this, so the refutation we get back from the proof system is at the wrong level of abstraction, verbose, and not directly relevant to us...

```python
def avg(l: Vec[S[N], int]) -> int:
    """ Computes the average of a nonempty list of ints. """
    return sum(l.l) // len(l.l)

avg(42) # error: # error: Argument of type "Literal[42]" cannot be assigned to parameter "l" of type "Vec[S[N@avg], int]" in function "avg"   "Literal[42]" is incompatible with "Vec[S[N@avg], int]"

avg(empty) # error: Argument of type "Vec[Z, int]" cannot be assigned to parameter "l" of type "Vec[S[N@avg], int]" in function "avg"; TypeVar "N@Vec" is covariant; "Z" is incompatible with "S[N@avg]"
```

## Non-solution 2: full dependent types

Previously, `Vec[N,T]` took two type-level arguments.  We could not parameterize it on a program term of type `int` (if we could, we could have avoided redefining the natural numbers in the type system.)

A type system that _does_ allow this is called a _dependent type system_.  

```haskell
constant vec : Type u → ℕ → Type u

-- notice that we can write 0 and (n + 1) directly! 
constant empty : Π α : Type u, vec α 0
constant cons :
  Π (α : Type u) (n : ℕ), α → vec α n → vec α (n + 1)
```

TODO: they are awesome except enjoy writing the proofs